**##############  Tools Setup  ##############**

In [1]:
import sklearn
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

# Importing the Keras libraries and packages
import kerastuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

**##############  Import datasets  ##############**

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

**##############  Preprocessing  ##############**

In [3]:
targets = train['y']
train = train.drop(columns=['y','w'])

**Data Scaling**(optional, better accuracy without)

**##############  Auto-Tune (does not work)  ##############**

In [4]:
X_tr, X_te, y_tr, y_te = train_test_split(train, targets, test_size=0.2,train_size=0.8)

In [5]:
X = train.values.reshape(7326,50,1)
y = targets.values.reshape(7326,1,1)
test = test.values.reshape(3141,50,1)

In [6]:
X_tr = X_tr.values.reshape(5860,50,1)
y_tr = y_tr.values.reshape(5860,1,1)
X_te = X_te.values.reshape(1466,50,1)
y_te = y_te.values.reshape(1466,1,1)

X_tr_sh, y_tr_sh = shuffle(X_tr, y_tr)
X_te_sh, y_te_sh = shuffle(X_te, y_te)

In [7]:
def build_model(hp):
    model = Sequential()
    
    model.add(LSTM(units=hp.Int('units',
                                        min_value=20,
                                        max_value=40,
                                        step=2),
                           return_sequences = True, input_shape = (X_tr.shape[1], 1)))
    model.add(Dropout(0.5))
    
    model.add(LSTM(units=hp.Int('units',
                                        min_value=4,
                                        max_value=16,
                                        step=2),
                           return_sequences = True))
    model.add(Dropout(0.3))
    
    model.add(LSTM(units=hp.Int('units',
                                        min_value=2,
                                        max_value=8,
                                        step=2),
                           ))
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error'])
    return model

In [8]:
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_error',
    max_epochs=50,
    hyperband_iterations=2,
    directory='Megaus')

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 2, 'sampling': None}
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001], 'ordered': True}


In [10]:
tuner.search(X_tr_sh,y_tr_sh,
             epochs=5,
             validation_data=(X_te_sh,y_te_sh))

Trial 59 Complete [00h 01m 43s]
val_mean_squared_error: 0.08281482756137848

Best val_mean_squared_error So Far: 0.01916494593024254
Total elapsed time: 01h 07m 23s
INFO:tensorflow:Oracle triggered exit


**##############  Build Regressor  ##############**

In [11]:
best = tuner.get_best_models(num_models=1)[0]

In [12]:
best.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 50, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'lstm_input'}},
  {'class_name': 'LSTM',
   'config': {'name': 'lstm',
    'trainable': True,
    'batch_input_shape': (None, 50, 1),
    'dtype': 'float32',
    'return_sequences': True,
    'return_state': False,
    'go_backwards': False,
    'stateful': False,
    'unroll': False,
    'time_major': False,
    'units': 22,
    'activation': 'tanh',
    'recurrent_activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'recurrent_initializer': {'class_name': 'Orthogonal',
     'config': {'gain': 1.0, 'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'unit_forget_bias': True,
    'kernel_regularizer': None,
    'recurrent_regularizer': None,
    'bias_regularizer': None,
    'activ

In [14]:
regressor = Sequential()
regressor.add(LSTM(units = 30, return_sequences = True, input_shape = (X_tr.shape[1], 1)))
regressor.add(Dropout(0.5))
regressor.add(LSTM(units = 10, return_sequences = True))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 5))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

**##############  Fit Regressor  ##############**

In [14]:
best.fit(X, y, epochs = 50, batch_size = 16)

Epoch 1/50
458/458 [==============================] - 16s 35ms/step - loss: 0.0330 - mean_squared_error: 0.0330
Epoch 2/50
458/458 [==============================] - 15s 32ms/step - loss: 0.0312 - mean_squared_error: 0.0312
Epoch 3/50
458/458 [==============================] - 15s 33ms/step - loss: 0.0329 - mean_squared_error: 0.0329
Epoch 4/50
426/458 [==========================>...] - ETA: 1s - loss: 0.0315 - mean_squared_error: 0.0315

KeyboardInterrupt: 

**##############  Test Predictions  ##############**

In [21]:
y_pred = best.predict(test)

**##############  Save File  ##############**

In [22]:
np.savetxt('RNNautotuned.txt', np.c_[y_pred])